In [1]:
%config IPCompleter.use_jedi=False

In [2]:
%%capture
!pip install openai
!pip install tiktoken

In [2]:
import os
import traceback

import pyodbc
import backoff
import chromadb
import pandas as pd
from loguru import logger

from chromadb.config import Settings

In [3]:
from bot import NewsBot

In [4]:
news_bot = NewsBot()

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Exception: {"error":"ValueError('Collection pocos-news-embedding does not exist.')"}

# Functions

In [4]:
# class InvalidCredentialsError(Exception):
#     def __init__(self):
#         error_message = "Invalid credentials."
#         super().__init__(error_message)

# @backoff.on_exception(
#     backoff.expo,
#     pyodbc.Error,
#     max_tries=5,
#     max_time=60,
# )
# def get_azure_sql_connection(connection_string: str, timeout: int):
#     try:
#         return pyodbc.connect(connection_string, timeout=timeout)
#     except pyodbc.InterfaceError:
#         raise InvalidCredentialsError()

In [5]:
# # Parameters

# timeout = 60

# table_name = f'{os.environ["RAW_DATA_SCHEMA"]}.{os.environ["RAW_DATA_TABLE"]}'

# connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Connection Timeout={timeout};'.format(
#     server = os.environ["AZURE_SQL_SERVER"],
#     database = os.environ["AZURE_SQL_DATABASE"],
#     driver = os.environ["AZURE_SQL_DRIVER"],
#     username = os.environ["AZURE_SQL_USERNAME"],
#     password = os.environ["AZURE_SQL_PASSWORD"],
#     timeout = timeout,
# )


# conn = get_azure_sql_connection(connection_string, timeout)


In [6]:
def _set_local_context(content, meta):
    return (
        f"<data>{meta['date']}</data>\n"
        f"<titulo>{meta['title']}</titulo>\n"
        f"<autor>{meta['author']}</autor>\n"
        f"<link>{meta['link']}</link>\n"
        f"<conteudo>{content}</conteudo>\n"
    )

def _set_query_content(documents, metadata):
    return"\n".join(
        sorted(
            [
                _set_local_context(content, meta) for content, meta in zip(documents, metadata)
            ]
        )
    )

def get_content(collection, query, n_results=10, n_neighbors=1000):

    result = predict(
        collection,
        query,
        n_results=n_results,
        n_neighbors=n_neighbors,
    )
    
    return _set_query_content(result['documents'][0], result['metadatas'][0])

def format_history_message(messages, human_prefix: str = 'Human', ai_prefix: str = 'AI'):
    for message in messages:
        if isinstance(message, langchain_core.messages.human.HumanMessage):
            yield f"{human_prefix}: {message.content}\n"
        
        elif isinstance(message, langchain_core.messages.ai.AIMessage):
            yield f"{ai_prefix}: {message.content}\n"

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.utils import embedding_functions


embedding_model = "clips/mfaq"
# embedding_model = "rufimelo/Legal-BERTimbau-sts-large"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

embedder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=embedding_model)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [8]:
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma

host_name = "chroma-server"
port = 8000
collection_name = 'pocos-news-embedding'
persist_directory = "chroma_db"

# --
settings = Settings(allow_reset=True, anonymized_telemetry=True, persist_directory=persist_directory)
chroma_client = chromadb.HttpClient(host=host_name, port = port, settings=settings)

vdb = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory=persist_directory,
    client_settings=settings,
    client=chroma_client,
    relevance_score_fn = lambda x, y: np.mean(x-y),
)

collection = chroma_client.get_collection(collection_name, embedding_function=embedder)

In [9]:
# Obtendo os documentos
docs = collection.get()

In [10]:
print("Exemplo de metadados:")
docs['metadatas'][0]

Exemplo de metadados:


{'author': 'ondapocos',
 'categories': 'Destaques|Esporte|Notícias',
 'date': '2021-08-08',
 'id': '000b577c-3fc6-5889-b0c4-96f88ca370e0',
 'link': 'https://ondapocos.com.br/olimpiadas-brasil-fica-em-melhor-posicao-da-historia-no-quadro-de-medalhas/',
 'snippet': 'O Brasil fez história nos Jogos Olímpicos de Tóquio ao conquistar 21 medalhas, ficando na 12ª colocação no quadro de medalhas, a melhor posição do...',
 'thumbnail_alt': 'OLIMPÍADAS | Brasil fica em melhor posição da história no quadro de medalhas',
 'thumbnail_link': 'https://i2.wp.com/ondapocos.com.br/wp-content/uploads/2021/08/ouro.jpg?resize=350%2C250&ssl=1',
 'title': 'OLIMPÍADAS | Brasil fica em melhor posição da história no quadro de medalhas'}

In [11]:
def predict(collection, query, n_neighbors: int = 1000, n_results: int = 10, **kwargs):

    def f(key, value, k):
        # print(key, value)
        def limit(x):
            if isinstance(x, list):
                return x[:k]
            return x
        if isinstance(value, list):
            return (key, [limit(x) for x in value])
        return (key, None)
    
    
    res = collection.query(
        query_texts=query,
        n_results=n_neighbors,
        **kwargs
        # where={"metadata_field": "is_equal_to_this"},
        # where_document={"$contains":"search_string"}
    )
      
    return  dict([f(key, value, n_results)  for key, value in res.items()])



In [12]:
predict(
    collection,
    "Mandado de prisão",
    n_results=2,
    n_neighbors=1000,
)

{'ids': [['314e354f-6d50-5634-a4ec-617efeb6e70d',
   '229b7783-948a-5340-b873-d573104a6683']],
 'distances': [[19.551469802856445, 19.70377540588379]],
 'embeddings': None,
 'metadatas': [[{'author': 'Aline Rodrigues',
    'categories': 'Destaques|Notícias|Policial',
    'date': '2022-10-05',
    'id': '314e354f-6d50-5634-a4ec-617efeb6e70d',
    'link': 'https://ondapocos.com.br/pai-acusado-de-estuprar-a-propria-filha-e-condenado-a-40-anos-de-prisao-no-sul-de-minas/',
    'snippet': 'A Justiça condenou a 40 anos e um mês de prisão, o homem acusado de estuprar a própria filha de 12 anos. A Ação Penal...',
    'thumbnail_alt': 'Pai acusado de estuprar a própria filha é condenado a 40 anos de prisão no Sul de Minas',
    'thumbnail_link': 'https://i0.wp.com/ondapocos.com.br/wp-content/uploads/2022/10/WhatsApp-Image-2022-10-05-at-09.06.12.jpeg?resize=350%2C250&ssl=1',
    'title': 'Pai acusado de estuprar a própria filha é condenado a 40 anos de prisão no Sul de Minas'},
   {'author': 'Ali

In [13]:
%config IPCompleter.use_jedi=False

In [14]:
from langchain.prompts import PromptTemplate

prompt_consulta_text = """
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado. Considere o CHAT HISTORY.

Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".

Nao considere nenhuma noticia que nao seja de poços de caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta
}}
Nao responda nada alem do JSON.

## CONTEXTO
{context}

## CHAT HISTORY
{history}

## HUMAN INPUT
{human_input}

IA:
"""

prompt = PromptTemplate(input_variables=["context", "history", "human_input"], template=prompt_consulta_text)

In [17]:
import langchain_core
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.llms import OpenAI

model_name: str = 'gpt-3.5-turbo'
temperature: float = 0.0

human_prefix: str = 'Human'
ai_prefix: str = 'AI'

llm = ChatOpenAI(model_name=model_name, temperature=temperature)

memory = ConversationSummaryBufferMemory(llm=OpenAI(temperature=0), return_messages=True, memory_key="chat_history", input_key="human_input", human_prefix=human_prefix, ai_prefix=ai_prefix)

chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

In [19]:
query = "Quais são as notícias mais recentes?"

context = get_content(collection, query, n_results=4, n_neighbors=1000)

_messages = memory.chat_memory.messages

history = "".join(format_history_message(_messages))

llm_response = chain.run(dict(context=context, history=history, human_input=query))



> Entering new LLMChain chain...
Prompt after formatting:

Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado. Considere o CHAT HISTORY.

Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".

Nao considere nenhuma noticia que nao seja de poços de caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta
}
Nao responda nada alem do JSON.

## CONTEXTO
<data>2020-04-02</data>
<titulo>Boletim sobre Covid-19 aponta que Poços continua com 178 casos suspeitos</titulo>
<autor>ondapocos</autor>
<link>https://ondapocos.com.br/boletim-sobre-covid-19-aponta-que-pocos-continua-com-178-casos-suspeitos/</link>
<conteudo>O Informe Epidemiológico divulgado pela Secretaria de Estado 

In [26]:
import ast

resp = ast.literal_eval(llm_response)

print(resp["resposta"])
print(resp["link"])

As notícias mais recentes são sobre a desativação do Painel Covid-19 de Poços de Caldas e a alteração na forma de divulgação dos dados da Covid-19 na cidade. Agora, os dados podem ser acessados na página da Secretaria de Estado de Saúde de Minas Gerais. O painel era atualizado diariamente, mas agora os dados serão atualizados semanalmente às quartas-feiras.
https://ondapocos.com.br/painel-covid-19-de-pocos-sera-desativado-dados-serao-atualizados-na-pagina-da-ses/


In [47]:
template_standalone_question = """
Voce vai receber uma pergunta de follow up da usuario que pode nao ter muitos detalhes.
Usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
Reescreva a frase do HUMAN INPUT para que ele possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
Não responde o usuário. Apenas reescreva a pergunta dele.

## CHAT HISTORY
{history}

## HUMAN INPUT
{human_input}

IA:
"""

prompt_standalone_question = PromptTemplate(input_variables=["history", "human_input"], template=template_standalone_question)



chain_standalone_question = LLMChain(llm=llm, prompt=prompt_standalone_question, verbose=True)


In [49]:
query = "Quero mais detalhes sobre isso"
chain_standalone_question.run(dict(history=history, human_input=query))



> Entering new LLMChain chain...
Prompt after formatting:

Voce vai receber uma pergunta de follow up da usuario que pode nao ter muitos detalhes.
Usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
Reescreva a frase do HUMAN INPUT para que ele possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
Não responde o usuário. Apenas reescreva a pergunta dele.

## CHAT HISTORY
Human: Quais são as notícias mais recentes?
AI: {
    "resposta": "As notícias mais recentes são sobre a desativação do Painel Covid-19 de Poços de Caldas e a alteração na forma de divulgação dos dados da Covid-19 na cidade. Agora, os dados podem ser acessados na página da Secretaria de Estado de Saúde de Minas Gerais. O painel era atualizado diariamente, mas agora os dados serão atualizados semanalmente às quartas-feiras.",
    "link": "https://ondapocos.com.br/painel-covid-19-de-pocos-sera-desativado-dados-serao-atualizados-na

'Quais são os detalhes adicionais sobre a desativação do Painel Covid-19 de Poços de Caldas e a alteração na forma de divulgação dos dados da Covid-19 na cidade?'

In [ ]:
query = "E as mais antigas"

In [89]:


type(_messages[1])

langchain_core.messages.ai.AIMessage

In [91]:
_messages[0]

HumanMessage(content='Quais são as notícias mais recentes?')

In [51]:
class Bot():

    def __init__(self):

        self.standalone_question = StandaloneBot()
        
        pass

    def perguntar(self, pergunta):

        pergunta_melhorada = self.standalone_question.improve()
        
        pass

    def responder(self):
        pass


bot = Bot()

In [54]:
bot.perguntar("Quais são as notícias mais recentes?")

In [76]:
import os
import ast
import traceback
import pyodbc
import backoff
import pandas as pd
import json

import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import langchain_core
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory
from langchain.llms import OpenAI
from langchain.prompts import load_prompt
from loguru import logger
import tiktoken


class NewsBot():

    def __init__(self):

        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
        self.llm_chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, model_kwargs={"stop": ["HUMAN_INPUT", "IA:"]})

        self.embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")
        self.embedder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="clips/mfaq")

        self.verbose_chains = True
        
        # Load utilitary prompts and chains
        self.prompt_intention = load_prompt("prompts/prompt_user_intention.json")
        self.prompt_standalone_question = load_prompt("prompts/prompt_standalone_question.json")
        
        # Load chat prompts and chains
        self.prompt_greeting = load_prompt("prompts/prompt_greeting.json")
        self.prompt_query = load_prompt("prompts/prompt_query.json")

        self.collection = self.get_chroma_collection()

    def get_chroma_collection(self):

        host_name = "chroma-server"
        port = 8000
        collection_name = 'pocos-news-embedding'
        persist_directory = "chroma_db"
        
        settings = Settings(allow_reset=True, anonymized_telemetry=True, persist_directory=persist_directory)
        chroma_client = chromadb.HttpClient(host=host_name, port = port, settings=settings)
        
        # vdb = Chroma(
        #     collection_name=collection_name,
        #     embedding_function=self.embeddings,
        #     persist_directory=persist_directory,
        #     client_settings=settings,
        #     client=chroma_client,
        #     relevance_score_fn = lambda x, y: np.mean(x-y),
        # )
        
        return chroma_client.get_collection(collection_name, embedding_function=self.embedder)
    
    def predict(self, collection, query, n_neighbors: int = 1000, n_results: int = 10, **kwargs):

        def f(key, value, k):
            # print(key, value)
            def limit(x):
                if isinstance(x, list):
                    return x[:k]
                return x
            if isinstance(value, list):
                return (key, [limit(x) for x in value])
            return (key, None)
        
        
        res = collection.query(
            query_texts=query,
            n_results=n_neighbors,
            **kwargs
            # where={"metadata_field": "is_equal_to_this"},
            # where_document={"$contains":"search_string"}
        )
      
        return  dict([f(key, value, n_results)  for key, value in res.items()])
    
    def _set_local_context(self, content, meta):
        return (
            f"<data>{meta['date']}</data>\n"
            f"<titulo>{meta['title']}</titulo>\n"
            f"<autor>{meta['author']}</autor>\n"
            f"<link>{meta['link']}</link>\n"
            f"<conteudo>{content}</conteudo>\n"
        )
    
    def count_tokens(self, context: str) -> int:
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        num_tokens = len(encoding.encode(context))
        
        return num_tokens
    
    def _set_query_content(self, documents, metadata):
        context_list = [self._set_local_context(content, meta) for content, meta in zip(documents, metadata)]

        tokens = 4000
        max_tokens = 3600
        while tokens > max_tokens:
            tokens = self.count_tokens("\n".join(context_list))
            if tokens >= max_tokens:
                context_list.pop()
        
        return "\n".join(sorted(context_list))
    
    def get_content(self, query, n_results=10, n_neighbors=1000):

        result = self.predict(
            self.collection,
            query,
            n_results=n_results,
            n_neighbors=n_neighbors,
        )
    
        return self._set_query_content(result['documents'][0], result['metadatas'][0])

    def format_history_message(self, messages, human_prefix: str = 'Human', ai_prefix: str = 'AI'):
        for message in messages:
            if isinstance(message, langchain_core.messages.human.HumanMessage):
                yield f"{human_prefix}: {message.content}\n"
            
            elif isinstance(message, langchain_core.messages.ai.AIMessage):
                yield f"{ai_prefix}: {message.content}\n"

    def get_standalone_question(self, message: str, *args, **kwargs) -> str:

        self.chain_standalone_question = LLMChain(
            llm=self.llm, prompt=self.prompt_standalone_question, verbose=self.verbose_chains
        )

        return self.chain_standalone_question.predict(history=kwargs["history"], human_input=message)
    
    def get_user_intention(self, message: str, *args, **kwargs) -> str:

        self.chain_intention = LLMChain(
            llm=self.llm, prompt=self.prompt_intention, verbose=self.verbose_chains
        )

        return self.chain_intention.predict(history=kwargs["history"], human_input=message)
    
    def handler_start_conversation(self, message: str, *args, **kwargs) -> str:

        self.chain_greeting = LLMChain(
            llm=self.llm_chat, prompt=self.prompt_greeting, verbose=self.verbose_chains, memory=kwargs["memory"]
        )

        return self.chain_greeting.predict(history=kwargs["history"], human_input=message)

    def handler_query(self, message: str, *args, **kwargs) -> str:

        self.chain_query = LLMChain(
            llm=self.llm_chat, prompt=self.prompt_query, verbose=self.verbose_chains, memory=kwargs["memory"]
        )

        context = self.get_content(query=message)

        response = self.chain_query.predict(history=kwargs["history"], human_input=message, context=context)
        resp_dict = ast.literal_eval(response)

        return f'{resp_dict["resposta"]}\n\nNotícia: {resp_dict["link"]}'

    def handler_fallback(self, *args, **kwargs) -> str:

        return (
            "Desculpe, mas não posso responder a essa pergunta. "
            "Algo em que possa ajudar sobre notícias de Poços de Caldas e região?"
        )

    def execute(self, message: str):

        memory = ConversationSummaryBufferMemory(
            llm=OpenAI(temperature=0),
            chat_history=ChatMessageHistory(),
            return_messages=True,
            memory_key="chat_history",
            input_key="human_input",
            human_prefix="Human",
            ai_prefix="AI"
        )
        chat_history = "".join(self.format_history_message(memory.chat_memory.messages))
        
        standalone_question = self.get_standalone_question(message=message, history=chat_history)
        print(f"Pergunta original: {message}")
        print(f"Pergunta melhorada: {standalone_question}")
        
        intention = self.get_user_intention(message=standalone_question, history=chat_history)
        print(f"Intenção: {intention}")
        
        handlers = {
            "inicio de conversa": self.handler_start_conversation,
            "consulta de conteudo": self.handler_query,
            "": self.handler_fallback
        }

        response = None
        for category, handler in handlers.items():
            if category in intention.lower().replace("ú", "u").replace("í", "i"):
                response = handler(
                    message=message,
                    memory=memory,
                    history=chat_history
                )
                break

        print(f">>> Resposta: {response}")
        
        return response


In [77]:
bot = NewsBot()

/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [78]:
bot.execute("olá")



> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY


## HUMAN INPUT
olá

IA:

> Finished chain.
Pergunta original: olá
Pergunta melhorada: Olá! Como posso ajudar você hoje?


> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconheca frases como "oi", "ola", "bom dia", "tudo bem:", etc.
- "Consulta de conteudo": Perguntas sobre no

'Olá! Como posso ajudar você hoje?'

In [79]:
bot.execute("Quais são as notícias mais recentes?")



> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY


## HUMAN INPUT
Quais são as notícias mais recentes?

IA:

> Finished chain.
Pergunta original: Quais são as notícias mais recentes?
Pergunta melhorada: Quais são as notícias mais recentes sobre Pocos de Caldas e região?


> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconhec

'As notícias mais recentes são sobre a desativação do Painel Covid-19 de Poços de Caldas e a alteração na forma de divulgação dos dados da Covid-19 na cidade. Agora, os dados podem ser acessados na página da Secretaria de Estado de Saúde de Minas Gerais. O painel de monitoramento de casos também pode ser acessado no link fornecido. O boletim será atualizado semanalmente às quartas-feiras.\n\nNotícia: https://ondapocos.com.br/painel-covid-19-de-pocos-sera-desativado-dados-serao-atualizados-na-pagina-da-ses/'

In [102]:


print()


In [73]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

llm = OpenAI(
    model="text-davinci-003",
    temperature=0.9,
    client="",
)

memory = ConversationBufferMemory(memory_key="chat_history")

prompt = PromptTemplate(
    input_variables=[
        "text_one",
        "text_two",
        "chat_history"
    ],
    template=(
        """You are an AI talking to a huamn. Here is the chat
        history so far:

        {chat_history}

        Here is some more text:

        {text_one}

        and here is a even more text:

        {text_two}
        """
    )
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)


output = chain.predict(
    text_one="Hello",
    text_two="World"
)

ValueError: One input key expected got ['text_one', 'text_two']

In [69]:


result['documents'][0]

['A Justiça condenou a 40 anos e um mês de prisão, o homem acusado de estuprar a própria filha de 12 anos. A Ação Penal foi proposta pelo Ministério Público de Minas Gerais, por meio da Promotoria de Justiça de Elói Mendes, onde ocorreu o caso. Segundo o Ministério Público, o réu já estava em prisão preventiva, no presídio local, e não poderá recorrer da sentença em liberdade. Conforme a sentença, o réu foi condenado pela prática, por 7 vezes, do crime de estupro de vulnerável, com aplicação das sanções do art. 217-A c/c art. 226, II, na forma do art. 71, todos do Código Penal. De acordo com a Ação Penal, proposta pelo promotor de Justiça Henrique Carlini Pereira, o homem praticava os abusos quando a mãe da vítima não estava em casa, e ameaçava bater na filha e “sumir com ela”, caso fosse denunciado. Um dos abusos teria sido presenciado por uma irmã da vítima. Os crimes ocorreram entre outubro de 2021 e fevereiro de 2022 e foram descobertos pela tia e pela avó paternas, que perceberam 

In [32]:
predict(
    collection,
    "Tráfico de drogas no bairro Nova Aurora",
    n_results=2,
    n_neighbors=1000,
    where={"categories": "Policial"},
    # where_document={"$contains":"search_string"},
)

{'ids': [['f57a2ac5-b14c-5f48-aa37-eba1039f4c35',
   'c8bd4ff5-56d9-5e8c-9442-a4553895ee23']],
 'distances': [[24.520042419433594, 32.876625061035156]],
 'embeddings': None,
 'metadatas': [[{'author': 'ondapocos',
    'categories': 'Policial',
    'date': '2020-08-28',
    'id': 'f57a2ac5-b14c-5f48-aa37-eba1039f4c35',
    'link': 'https://ondapocos.com.br/dupla-e-detida-com-drogas-e-radio-na-frequencia-da-pm/',
    'snippet': 'Um homem foi preso e um adolescente apreendido por tráfico de drogas, nesta quinta-feira,27. A dupla foi detida nesta quinta-feira,27, no bairro Jardim Country em...',
    'thumbnail_alt': 'Dupla é detida com drogas e rádio na frequência da PM',
    'thumbnail_link': 'https://i2.wp.com/ondapocos.com.br/wp-content/uploads/2020/08/tráfico-country-e-ottoni.jpg?resize=350%2C250&ssl=1',
    'title': 'Dupla é detida com drogas e rádio na frequência da PM'},
   {'author': 'andrevince',
    'categories': 'Policial',
    'date': '2022-02-21',
    'id': 'c8bd4ff5-56d9-5e8c

In [46]:
predict(
    collection,
    "Assalto",
    n_results=10,
    n_neighbors=1000,
    # where={"categories": "Policial"},
    where_document={"$contains":"Jardim Formosa"},
)

{'ids': [['00f25c5b-d824-54bd-8f28-a75c441bb796',
   '5ad81b66-aaa6-59e4-be73-57a477a39f7e',
   'd8937373-e227-5590-8c61-ddde003027db',
   '8d6fdbb9-9c80-560b-95dd-268679e57eb9',
   'f765c1d8-ac9d-5aa8-b3e5-7e5d6d26bb8a',
   '2ed146e6-e581-53d1-abb1-a10ed6603a3a',
   'ba0e1d16-3ac6-5986-aaab-cacdcfdc6c08',
   'c47656d5-4e5f-5253-8c96-f33b336e4dec',
   '5d51ba1c-66b0-5f95-9ea4-852b9b9a17e7',
   '25923b51-37cd-5b8d-8949-9c6d452a7e05']],
 'distances': [[27.759767532348633,
   29.31196403503418,
   29.640850067138672,
   29.723114013671875,
   29.862071990966797,
   30.115741729736328,
   30.52475929260254,
   31.70189094543457,
   31.853361129760742,
   32.88178253173828]],
 'embeddings': None,
 'metadatas': [[{'author': 'ondapocos',
    'categories': 'Notícias',
    'date': '2020-10-09',
    'id': '00f25c5b-d824-54bd-8f28-a75c441bb796',
    'link': 'https://ondapocos.com.br/utilidade-publica-voce-viu-a-juju-ela-esta-desaparecida-em-pocos-de-caldas/',
    'snippet': 'A Calopsita da foto é

# 